# Convolutional Neural Networks with Keras (3)

### Fine-tuning the top layers of a a pre-trained network

We can try to "fine-tune" the last convolutional block of the VGG16 model alongside the top-level classifier.

Fine-tuning consist in starting from a trained network, then re-training it on a new dataset using very small weight updates. 

    


Steps:

1) instantiate the convolutional base of VGG16 and load its weights

2) add our previously defined fully-connected model on top, and load its weights

3) freeze the layers of the VGG16 model up to the last convolutional block

In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import numpy as np

Using TensorFlow backend.


In [3]:
img_width, img_height = 375,500

train_data_dir = 'data/train'
test_data_dir = 'data/test'

n_train_samples = 235
n_train_ants = 114
n_train_bees = 121

n_test_samples = 148
n_test_ants = 66
n_test_bees = 82

num_epochs = 50
batch_size = 16

top_model_weights_path = 'bottleneck_fc_model.h5'

In [4]:
# load the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

In [6]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

ValueError: The shape of the input to "Flatten" is not fully defined (got (None, None, 512). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.

In [7]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

OSError: Unable to open file (Unable to open file: name = 'bottleneck_fc_model.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
# add the model on top of the convolutional base
model.add(top_model)

In [ ]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
layer.trainable = False

In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
metrics=['accuracy'])

In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')


In [ ]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=n_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=n_validation_samples)

### test the model

In [ ]:
def score_img(img_path):
    img = load_img(img_path, grayscale=True) 
    candidate = np.expand_dims(img_to_array(img)/255, axis=0)
    pred = test_model.predict_classes(candidate, verbose=0)
    return pred[0,0]

In [ ]:
cracks_total = 837
num_cracks_found = 0
for imgfile in os.listdir('data/test/crack'):
    score = score_img('data/test/crack/' + imgfile)
    if score == 0: num_cracks_found = num_cracks_found + 1
print('Found crack in {} of {} images'.format(num_cracks_found, cracks_total))

In [ ]:
no_cracks_total = 772
num_no_cracks_found = 0
for imgfile in os.listdir('data/test/nocrack'):
    score = score_img('data/test/nocrack/' + imgfile)
    if score == 1: num_no_cracks_found = num_no_cracks_found + 1
print('Correctly said "no crack" in {} of {} images'.format(num_no_cracks_found, no_cracks_total))

In [ ]:
true_positives = num_cracks_found
false_negatives = cracks_total - true_positives
true_negatives = num_no_cracks_found
false_positives = no_cracks_total - true_negatives

accuracy = (true_positives + true_negatives) / (cracks_total + no_cracks_total)
sensitivity = true_positives / (true_positives + false_negatives)
precision = true_positives / (true_positives + false_positives)

print('Accuracy: {:.3f}, sensitivity: {:.3f}, precision: {:.3f}'.format(accuracy, sensitivity, precision))